In [ ]:
# !pip install --user --upgrade pip

In [ ]:
# !pip install --user --upgrade bitsandbytes peft trl datasets transformers accelerate

# Import Packages & Set All Parameters

## Import Packages

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## Source of The Model & Dataset

In [ ]:
# The model that you want to train from the Hugging Face hub
# For Llama
model_type = 'llama'
model_name = "NousResearch/Llama-2-7b-chat-hf"
model_token = None
new_model = "Llama2-7b-chat-finetune"

# # For Gemma model
# model_type = 'gemma'
# model_name = "google/gemma-2b"
# model_token = 'GEMMA_TOKEN' 
# new_model = "Gemma-2b-chat-finetune"

# The instruction dataset to use
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"

## Parameters for Quantization

In [ ]:
# LoRA attention dimension
lora_r = 8

# Alpha parameter for LoRA scaling
lora_alpha = 12

# Dropout probability for LoRA layers
lora_dropout = 0.25

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

## Training Parameters

In [ ]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = 100

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 5

# Maximum sequence length to use
max_seq_length = 500

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Load & Process The Dataset 

In [ ]:
# Load dataset
dataset = load_dataset(dataset_name, split="train").shuffle(seed=42).select(range(1000))

In [ ]:
print(dataset)

In [ ]:
context = f"""You are a helpful assistant that generate codes in Python to do a certain thing.
            \nYour task is to follow the given instruction that describes a task. Write a response that appropriately completes the task.
            \nThere might be an input provided, if so, utilize the input to complete the task.
            \nBelow is the instruction and input."""
    
def transform_dataset_into_proper_format(row):
    instruction = row['instruction']
    input_val = row['input']
    output_val = row['output']
    
    if model_type == 'llama':
        reformatted_prompt = f"""<s>[INST]\n{context}\n\nInstruction:\n{instruction}\n\nInput:\n{input_val}\n[/INST]\n\nOutput:\n{output_val}\n</s>"""
        
    elif model_type == 'gemma':
        reformatted_prompt = f"""<start_of_turn>User\n{context}\n\nInstruction:\n{instruction}\n\nInput:\n{input_val}\n<end_of_turn>\n<start_of_turn>Model\n\n{output_val}<end_of_turn>"""
    
    return {'prompt': reformatted_prompt}

transformed_dataset = dataset.map(transform_dataset_into_proper_format)

In [ ]:
print(transformed_dataset[5]['prompt'])

In [ ]:
split_transformed_dataset = transformed_dataset.train_test_split(seed=42, test_size=0.01)

# Quantize & Fine-Tune LLM

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    token=model_token
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=split_transformed_dataset['train'],
    peft_config=peft_config,
    dataset_text_field="prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

# Train model
trainer.train()

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

# Playing Around with The Fine-Tuned LLM

In [ ]:
def make_inference(index, max_length_val):
    # Ignore warnings
    logging.set_verbosity(logging.CRITICAL)
    
    instruction = split_transformed_dataset['test'][index]['instruction']
    input_val = split_transformed_dataset['test'][index]['input']
    
    # Run text generation pipeline with our next model
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length_val)

    if model_type == 'llama':
        reformatted_prompt = f"""<s>[INST]\n{context}\n\nInstruction:\n{instruction}\n\nInput:\n{input_val}\n[/INST]\n\nOutput:\n"""

    elif model_type == 'gemma':
        reformatted_prompt = f"""<start_of_turn>User\n{context}\n\nInstruction:\n{instruction}\n\nInput:\n{input_val}\n<end_of_turn>\n<start_of_turn>Model\n\n"""

    result = pipe(reformatted_prompt)
    
    return result

In [ ]:
result = make_inference(2, 190)
print(result[0]['generated_text'])

In [ ]:
result = make_inference(6, 260)
print(result[0]['generated_text'])

# Share The Fine-Tuned LLM to The HuggingFace

In [ ]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from huggingface_hub import login
login()

In [ ]:
huggingface_name = 'USERNAME'
model.push_to_hub(f"{huggingface_name}/create_python_code_from_instruction_{model_type}", check_pr=True)
tokenizer.push_to_hub(f"{huggingface_name}/create_python_code_from_instruction_{model_type}",check_pr=True)